In [7]:
import csv
from sklearn import cross_validation

In [8]:
title_tagged_file = open("/Users/raghavan/Downloads/source_file/final_input.txt",'rb')
title_tags = csv.reader(title_tagged_file, delimiter='\t')

In [9]:
titles_key_values_groupedby = dict()

In [10]:
for i in title_tags:
    if i[0] not in titles_key_values_groupedby.keys():
        titles_key_values_groupedby[i[0]] = []
    titles_key_values_groupedby[i[0]].append(i[1:3])

In [11]:
['Old Path White Clouds', 'size', 'white', '2', '2', '1', '1', '4', '246'][1:3]

['size', 'white']

In [12]:
titles_key_values_with_flattend_tags = dict()

In [13]:
for key,value in titles_key_values_groupedby.iteritems():
    tags = {}
    for i in value:
        for j in i[1].split(" "):
            tags[j] = i[0]
    titles_key_values_with_flattend_tags[key] = tags       

In [14]:
# titles_key_values_with_flattend_tags

In [15]:
tokenised_title_tagging = {}
for key , value in titles_key_values_with_flattend_tags.iteritems():
    taggings = []
    for token in key.split(" "):
        if token.lower() in value.keys():
            taggings.append((token.lower(),value[token.lower()].replace(" ","_")))
        else: 
            taggings.append((token,'other'))
    tokenised_title_tagging[key] = taggings

In [16]:
# tokenised_title_tagging

In [56]:
output_file = open("/Users/raghavan/Downloads/sample_input_file.txt",'w')
for title , key_value in tokenised_title_tagging.iteritems():
    for combo in key_value:
        output_file.write(combo[0] + "\t" + combo[1] + '\n')
    output_file.write('\n')    
output_file.close()    

In [18]:
total_base_data = []
for title , key_value in tokenised_title_tagging.iteritems():
    training_line = []
    for combo in key_value:
        training_line.append(combo)
    total_base_data.append(training_line)       

In [37]:
training_data = total_base_data[:5500]
test_data = total_base_data[5501:]

In [38]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [39]:
%%time
X_train = [sent2features(s) for s in training_data]
y_train = [sent2labels(s) for s in training_data]

X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]

CPU times: user 639 ms, sys: 56.6 ms, total: 695 ms
Wall time: 688 ms


In [40]:
X_train[0]

[['bias',
  'word.lower=hewlett',
  'word[-3:]=ett',
  'word[-2:]=tt',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'BOS',
  '+1:word.lower=packard',
  '+1:word.istitle=False',
  '+1:word.isupper=False'],
 ['bias',
  'word.lower=packard',
  'word[-3:]=ard',
  'word[-2:]=rd',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  '-1:word.lower=hewlett',
  '-1:word.istitle=False',
  '-1:word.isupper=False',
  '+1:word.lower=enterprise',
  '+1:word.istitle=False',
  '+1:word.isupper=False'],
 ['bias',
  'word.lower=enterprise',
  'word[-3:]=ise',
  'word[-2:]=se',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  '-1:word.lower=packard',
  '-1:word.istitle=False',
  '-1:word.isupper=False',
  '+1:word.lower=hpe',
  '+1:word.istitle=False',
  '+1:word.isupper=True'],
 ['bias',
  'word.lower=hpe',
  'word[-3:]=HPE',
  'word[-2:]=PE',
  'word.isupper=True',
  'word.istitle=False',
  'word.isdigit=False',
  '-1:word.lowe

In [41]:
import pycrfsuite

In [42]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 675 ms, sys: 16 ms, total: 691 ms
Wall time: 687 ms


In [43]:

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [44]:
trainer.params()


['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [45]:
trainer.train('conll2002-esp.crfsuite')

In [46]:
trainer.logparser.last_iteration

{'active_features': 2376,
 'error_norm': 41.127353,
 'feature_norm': 85.482605,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 5476.865797,
 'num': 100,
 'scores': {},
 'time': 0.392}

In [47]:
print len(trainer.logparser.iterations), trainer.logparser.iterations[-1]


100 {'loss': 5476.865797, 'error_norm': 41.127353, 'linesearch_trials': 1, 'active_features': 2376, 'num': 100, 'time': 0.392, 'scores': {}, 'linesearch_step': 1.0, 'feature_norm': 85.482605}


In [48]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r--  1 raghavan  staff   133K Apr 18 17:30 ./conll2002-esp.crfsuite


In [49]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

In [54]:
spot_checking = [("token", "predicted", "actual")]
for example_sent in test_data:
    for i in range(len(sent2tokens(example_sent))):
        spot_checking.append(((sent2tokens(example_sent)[i]), 
                              tagger.tag(sent2features(example_sent))[i], 
                              sent2labels(example_sent)[i]))

output_file = open("/Users/raghavan/Downloads/spot_check_18920.tsv",'w')
for token, predicted, actual in spot_checking:
    output_file.write(token + "\t" + predicted + "\t" + actual + '\n')
output_file.close()

In [50]:
test_data
# example_sent = test_data[0]
# print ' '.join(sent2tokens(example_sent))

# print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
# print("Correct:  ", ' '.join(sent2labels(example_sent)))

[[('hewlett', 'cpu_series'),
  ('packard', 'cpu_series'),
  ('enterprise', 'cpu_series'),
  ('HPE', 'other'),
  ('5Y', 'other'),
  ('PCA', 'other'),
  ('24X7', 'other'),
  ('VMW', 'other'),
  ('VSPH', 'other'),
  ('2XENTPLUS', 'other'),
  ('SVC', 'other'),
  ('(U6GS1E)', 'other')],
 [('Simply', 'other'),
  ('nuc', 'cpu_series'),
  ('nuc', 'cpu_series'),
  ('5', 'other'),
  ('I3', 'other'),
  ('4gb', 'general_warranty'),
  ('128GB', 'other'),
  ('no', 'operating_system'),
  ('os', 'operating_system'),
  ('(910-8C10-011)', 'other')],
 [('system', 'package_contents'),
  ('Design', 'other'),
  ('Advantage', 'other'),
  ('NEW-BEZEL', 'other'),
  ('SERVICE', 'other'),
  ('ASM', 'other'),
  ('PIAZZA', 'other'),
  ('(754736-001)', 'other')],
 [('hewlett', 'cpu_series'),
  ('packard', 'cpu_series'),
  ('enterprise', 'cpu_series'),
  ('HPE', 'other'),
  ('1Y', 'other'),
  ('FC', 'other'),
  ('24X7', 'other'),
  ('MSM720', 'other'),
  ('ACCESS', 'other'),
  ('CONTR', 'other'),
  ('SVC', 'other'),

In [51]:
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )


In [52]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 188 ms, sys: 4.64 ms, total: 193 ms
Wall time: 193 ms


In [53]:
print(bio_classification_report(y_test, y_pred))

                                   precision    recall  f1-score   support

              3d_glasses_included       1.00      0.93      0.96        14
                         best_use       1.00      1.00      1.00        17
               cd_read_speed_max.       0.91      0.95      0.93        83
                            color       0.94      0.92      0.93        97
                        condition       1.00      1.00      1.00        77
                   cpu_cache_(mb)       0.00      0.00      0.00         2
                         cpu_core       0.80      0.92      0.86        13
                       cpu_series       0.98      0.98      0.98      3135
                        cpu_speed       0.96      0.86      0.91        93
                      depth_(in.)       0.00      0.00      0.00         5
                      form_factor       0.94      0.85      0.90       317
                 general_warranty       0.87      0.97      0.92       247
                      gp

/Users/raghavan/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
